In [1]:
import pandas as pd
import numpy as np
import os, re
from datetime import datetime
import librosa
import gc

In [2]:
def extract_number(filename):
    numbers = re.findall(r'\d+', filename)
    return [int(num) for num in numbers]
from collections import defaultdict

def split_files_by_first_two_elements(file_list):
    split_files = defaultdict(list)
    for file in file_list:
        parts = file.split('_')
        date_obj = datetime.strptime(parts[0], '%Y%m%d')
        if date_obj > datetime.strptime('20230902','%Y%m%d') or  date_obj < datetime.strptime('20230823', '%Y%m%d'):
            continue
        if len(parts) >= 2:
            key = f"{parts[0]}_{parts[1]}_{parts[2]}"  # 첫 번째와 두 번째 요소를 결합
            split_files[key].append(file)
    return split_files

In [3]:
# 데이터프레임을 이 key별로 합치기
def spliting(dataframe:pd.DataFrame):
    """
    데이터프레임에서 마지막 부분을 떼어내는 과정 
    """
    Times = dataframe['Time'].dt.floor('S').value_counts()
    Times = Times.sort_index(ascending=False).index[0]
    Last = dataframe[dataframe['Time'].dt.floor('S')==pd.Timestamp(Times)]
    if len(Last) < 100*2:
        Last = pd.DataFrame()
    return dataframe, Last

In [4]:
def concating_first(key:str,List:list):
    '''
    데이터프레임을 하나도 합치는 코드
    첫 시작할때하는 코드 
    '''
    dirs = r'D:\Acceleration'
    dataframe = pd.DataFrame()
    for name in List:
        data = pd.read_csv(os.path.join(dirs,name),skiprows=9) if 'csv' in name else pd.DataFrame()
        dataframe = pd.concat([dataframe,data],axis=0)
        del data
        gc.collect()
    # 다 했으니 마지막 초 부분은 split하기 
    dataframe['Time'] = pd.to_datetime(dataframe['Time'])
    dataframe, splited = spliting(dataframe) 
    return dataframe, splited

def concating(key:str,List:list,First:pd.DataFrame):
    '''
    데이터프레임을 하나도 합치는 코드
    그 이후 코드 
    '''
    dirs = r'D:\Acceleration'
    dataframe = First
    for name in List:
        data = pd.read_csv(os.path.join(dirs,name),skiprows=9) if 'csv' in name else pd.DataFrame()
        dataframe = pd.concat([dataframe,data],axis=0)
        del data
        gc.collect()
    # 다 했으니 마지막 초 부분은 split하기 
    dataframe['Time'] = pd.to_datetime(dataframe['Time'])
    dataframe, splited = spliting(dataframe) 
    return dataframe, splited

def concating_last(key:str,List:list,First:pd.DataFrame):
    '''
    데이터프레임을 하나도 합치는 코드
    마지막 코드 
    '''
    dirs = r'D:\Acceleration'
    dataframe = First
    for name in List:
        data = pd.read_csv(os.path.join(dirs,name),skiprows=9) if 'csv' in name else pd.DataFrame()
        dataframe = pd.concat([dataframe,data],axis=0)
        
    # 다 했으니 마지막 초 부분은 버리기
    dataframe['Time'] = pd.to_datetime(dataframe['Time'])
    dataframe, splited = spliting(dataframe)
    return dataframe



In [5]:
def adjust_data_length_per_time(df, target_length=10000):
    """
    데이터프레임을 'Time'으로 그룹화하고, 각 그룹의 데이터 길이를 target_length에 맞춥니다.

    Parameters:
    df (pd.DataFrame): 원본 데이터프레임
    target_length (int): 각 그룹의 타겟 데이터 길이 (기본값: 10000)

    Returns:
    pd.DataFrame: 조정된 데이터 길이를 갖는 새로운 데이터프레임, 'Time' 열 포함
    """
    adjusted_data = []

    # 'Time' 열을 초 단위로 내림 처리
    df['Time'] = pd.to_datetime(df['Time']).dt.floor('S')
    for time, group in df.groupby('Time'):
        # 데이터 길이 조정
        length = len(group)
        if length < target_length:
            # 데이터가 부족한 경우 0으로 채움
            fill_data = pd.DataFrame(np.zeros((target_length - length, 3),dtype=np.float64), columns=['x', 'y', 'z'])
            fill_data['Time'] = time
            group = pd.concat([group, fill_data], ignore_index=True)
        elif length > target_length:
            # 데이터가 너무 많은 경우 잘라냄
            group = group.iloc[:target_length]

        adjusted_data.append(group)

    # 모든 조정된 데이터를 하나의 데이터프레임으로 합침
    return pd.concat(adjusted_data, ignore_index=True)

In [6]:
def calculate_mfcc_per_second(data, sr=10000, n_mfcc=20, n_fft=2048, hop_length=1024):
    """
    1초 간격의 오디오 데이터에 대한 MFCC를 계산합니다.

    Parameters:
    data (numpy.ndarray): 오디오 데이터
    sr (int): 샘플링 빈도
    n_mfcc (int): 추출할 MFCC 계수의 수
    n_fft (int): FFT 윈도우의 크기
    hop_length (int): 프레임 간의 이동 스텝 크기

    Returns:
    pd.DataFrame: 각 1초 간격의 MFCC 계산 결과
    """
    mfccs_results = []

    # 데이터를 1초 간격으로 분할
    for i in range(0, len(data), sr):
        one_second_data = data[i:i+sr]
        if len(one_second_data) < sr:
            break
        mfccs = librosa.feature.mfcc(y=one_second_data, sr=sr, n_mfcc=n_mfcc, n_fft=n_fft, hop_length=hop_length)
        mfccs_mean = np.mean(mfccs, axis=1)  # MFCC의 평균을 계산
        mfccs_results.append(mfccs_mean)
    return pd.DataFrame(mfccs_results)

def mfcc_all(df:pd.DataFrame):
    Times = df['Time'].unique()
    mfcc_list = []
    for column in ['x','y','z']:
        temp = calculate_mfcc_per_second(np.array(df[column]),sr=10000)
        temp.index = Times
        mfcc_list.append(temp)
    return mfcc_list

### stft 및 저장

In [7]:
# 파일 목록을 숫자에 따라 정렬
files = os.listdir(r'D:\Acceleration')
files.sort(key=extract_number)
groups = split_files_by_first_two_elements(files)
print(len(groups))

128


In [8]:
lengths = {key: len(value) for key, value in groups.items()}

# 데이터프레임으로 변환
pd.DataFrame(list(lengths.items()), columns=['Key', 'List Length']).min()

Key            20230823_010900_866
List Length                     38
dtype: object

In [9]:
# 리스트에서 중복된 값의 개수를 계산
counts = pd.Series(files).value_counts()

# 데이터프레임으로 변환
pd.DataFrame({'Item': counts.index, 'Count': counts.values}).max()

Item     20230904_132518_259_9.csv
Count                            1
dtype: object

In [10]:
for i, (key, List) in enumerate(groups.items()):
    
    con = 21
    if i+1<con:
        print(i+1,end=' ')
        if (i+1)%10 ==0:
            print()
        continue

    times = datetime.now()
    # 데이터 프레임 합치기
    if i+1==con: # 39번째는 마지막 reamin만 가져오기 
        dataframe, remain = concating_first(key,List)
        print()
        print(f'{con}번째 그룹 remain load',f'걸린시간: {datetime.now()-times}')
        continue

    elif i==len(groups)-1:
        dataframe = concating_last(key,List, remain)
    else:
        dataframe,remain = concating(key,List,remain)
    print(f'{i+1}번째 그룹{len(List)}개 데이터 load 완료',f'걸린시간: {datetime.now()-times}')

    if len(remain)>20000:
        raise NotImplementedError
    # 데이터 복구 및 설정
    times = datetime.now()
    dataframe_re = adjust_data_length_per_time(dataframe,target_length=10000)
    results = mfcc_all(dataframe_re) #초마다 하나씩 MFCC 진행 
    
    #저장
    for j,column in enumerate(['x','y','z']): 
        name = f'{i+1}_{column}.csv'
        path = os.path.join('./raw_signal_new',name)
        results[j].to_csv(path)
    print(f'{i+1}번째 그룹 End', f'걸린시간: {datetime.now()-times}')
    del dataframe
    del results
    print()

1 2 3 4 5 6 7 8 9 10 
11 12 13 14 15 16 17 18 
19번째 그룹 remain load 걸린시간: 0:01:59.913833
20번째 그룹71개 데이터 load 완료 걸린시간: 0:02:20.703660
20번째 그룹 End 걸린시간: 0:01:42.924644

21번째 그룹71개 데이터 load 완료 걸린시간: 0:02:10.424588
21번째 그룹 End 걸린시간: 0:01:45.282701



KeyboardInterrupt: 